In [1]:
import torch
import torchvision
import torchvision.datasets as datasets
import torchvision.transforms as transforms
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import numpy as np
import random

In [2]:
device=torch.device('cude' if torch.cuda.is_available() else 'cpu') 
device

device(type='cpu')

In [3]:
#1D miniCNN
class MiniCNN(nn.Module):
    def __init__(self, input_channels, output_classes):
        super(MiniCNN, self).__init__()
        self.conv1 = nn.Conv1d(in_channels=input_channels, out_channels=16, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv1d(in_channels=16, out_channels=32, kernel_size=3, stride=1, padding=1)
        self.fc1 = nn.Linear(32 * 200, 128)  # Adjust input size based on your sequence length
        self.fc2 = nn.Linear(128, output_classes)
        
    def forward(self, x):
        # Apply convolutional layers
        x = F.relu(self.conv1(x))
        x = F.relu(self.conv2(x))
        
        # Flatten the output for fully connected layers
        x = x.view(x.size(0), -1)
        
        # Apply fully connected layers
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

In [4]:
class oneDAlexNet(nn.Module):
    def __init__(self, input_channels, num_classes):
        super(oneDAlexNet, self).__init__()
        self.features = nn.Sequential(
            nn.Conv1d(input_channels, 64, kernel_size=11, stride=4, padding=2),
            nn.ReLU(inplace=True),
            nn.MaxPool1d(kernel_size=3, stride=2),
            nn.Conv1d(64, 192, kernel_size=5, padding=2),
            nn.ReLU(inplace=True),
            nn.MaxPool1d(kernel_size=3, stride=2),
            nn.Conv1d(192, 384, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv1d(384, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv1d(256, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool1d(kernel_size=3, stride=2),
        )
        self.avgpool = nn.AdaptiveAvgPool1d(6)
        self.classifier = nn.Sequential(
            nn.Dropout(),
            nn.Linear(256 * 6, 4096),
            nn.ReLU(inplace=True),
            nn.Dropout(),
            nn.Linear(4096, 4096),
            nn.ReLU(inplace=True),
            nn.Linear(4096, num_classes),
        )

    def forward(self, x):
        x = self.features(x)
        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.classifier(x)
        return x
# 1DAlexNet will run with other function calls

In [5]:
#taken from alexnet example, nees to be formatted for oneDAlexnet
def train(model, device, train_loader, optimizer, loss_fn, epochs):
    model.train()
    for epoch in range(epochs):
        for batch_ids, (img, classes) in enumerate(train_loader):
            classes = classes.type(torch.LongTensor)
            img, classes = img.to(device), classes.to(device)
            optimizer.zero_grad()
            output = model(img)
            loss = loss_fn(output, classes)
            loss.backward()
            optimizer.step()
            if (batch_ids + 1) % 2 == 0:
                print("Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}".format(
                    epoch+1, batch_ids * len(img), len(train_loader.dataset),
                    100. * batch_ids / len(train_loader), loss.item()))

In [6]:
#taken from alexnet example needs to be formatted for oneDAlexnet
def test(model, device, test_loader, loss_fn):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for img, classes in test_loader:
            img, classes = img.to(device), classes.to(device)
            output = model(img)
            test_loss += loss_fn(output, classes, reduction='sum').item()
            _, predicted = torch.max(output, 1)
            correct += (predicted == classes).sum().item()
    test_loss /= len(test_loader.dataset)
    print("\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n".format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [7]:
#For testing with only 1000 lines, 
#Read the first 1000 lines from merged_data.txt
with open('merged_data.txt', 'r') as file:
    lines = [next(file).strip() for _ in range(1000)]

# Write the lines to a new file named 1000.txt
with open('1000.txt', 'w') as outfile:
    for line in lines:
        # Split the line into label and sequence
        label, sequence = line.split(' ', 1)
        # Write the label and sequence on the same line, separated by a space
        outfile.write(f"{label} {sequence}\n")


In [8]:
import random

def split_data(input_file, train_file, test_file, val_file, train_percent=0.75, test_percent=0.20):
    # Read the input data file
    with open(input_file, 'r') as f:
        data = f.readlines()

    # Calculate the number of rows for each split
    total_rows = len(data)
    print("totalrows:", total_rows)
    train_rows = int(train_percent * total_rows)
    print("training rows:", train_rows)
    test_rows = int(test_percent * total_rows)
    val_rows = total_rows - train_rows - test_rows

    # Randomly shuffle the data
    random.shuffle(data)

    # Write data to train, test, and validation files
    with open(train_file, 'w') as f:
        f.writelines(data[:train_rows])

    with open(test_file, 'w') as f:
        f.writelines(data[train_rows:train_rows + test_rows])

    with open(val_file, 'w') as f:
        f.writelines(data[train_rows + test_rows:])

# File paths
input_file = "merged_data.txt" #testing performed with 1000.txt
train_file = "train_data.txt"
test_file = "test_data.txt"
val_file = "val_data.txt"

# Split data into train, test, and validation files
split_data(input_file, train_file, test_file, val_file, train_percent=0.75, test_percent=0.20) #values can be changed to 80 train, and 20 test


totalrows: 641810
training rows: 481357


In [9]:
#one-hot-encoder function
def one_hot_encoder(sequence):
    sequence = sequence.upper()  # Convert sequence to uppercase so they all match
    mapping = {'A': 0, 'C': 1, 'G': 2, 'T': 3}
    one_hot = np.zeros((len(sequence), 4), dtype=int)
    for i, nucleotide in enumerate(sequence):
        if nucleotide in mapping:
            one_hot[i, mapping[nucleotide]] = 1
    return one_hot


In [10]:
#Read sequences from a data file and apply one-hot encoding
def process_data_file(input_file, output_file):
    data = []
    min_length = float('inf')  # Initialize with a large value
    
    # Read the input data file
    with open(input_file, 'r') as file:
        for line in file:
            parts = line.strip().split(' ')
            if len(parts) >= 2:
                label, sequence = parts[0], ' '.join(parts[1:])
                sequence = sequence.upper()  # Convert sequence to uppercase
                min_length = min(min_length, len(sequence))  # Update min_length
                data.append((int(label), sequence))
    
    # Crop sequences to the minimum length
    cropped_data = [(label, sequence[:min_length]) for label, sequence in data]
    
    # Write cropped data to the output file
    with open(output_file, 'w') as file:
        for label, sequence in cropped_data:
            file.write(f"{label} {sequence}\n")
    
    return cropped_data  # Return the cropped data



In [11]:
#custom dataset class
class CustomDataset(torch.utils.data.Dataset):
    def __init__(self, data):
        self.data = data

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        label, sequence = self.data[idx]
        # Convert sequence to one-hot encoded tensor
        sequence_tensor = torch.tensor(one_hot_encoder(sequence), dtype=torch.float32)
        label_tensor = torch.tensor(label, dtype=torch.float32)
        return sequence_tensor, label_tensor

In [12]:
#convert data to tensor fromat
def convert_to_tensor(dataset):
    tensor_data = []
    for sequence_tensor, label_tensor in dataset:
        tensor_data.append((sequence_tensor, label_tensor))
    return tensor_data

In [13]:
#data loader after tensor converion
def create_data_loader(dataset, batch_size, shuffle=True):
    return DataLoader(dataset, batch_size=batch_size, shuffle=shuffle)

In [14]:
#save data back to file
def save_data_to_file(dataset, file_path):
    with open(file_path, 'w') as file:
        for sequence_tensor, label_tensor in dataset:
            label = int(label_tensor.item())
            sequence_str = ''.join([['A', 'C', 'G', 'T'][i] for i in sequence_tensor.argmax(dim=1)])
            file.write(f"{label} {sequence_str}\n")

In [15]:
train_data = process_data_file('train_data.txt', "train_data_after_encode.txt")
train_dataset = CustomDataset(train_data)
train_dataset_tensor = convert_to_tensor(train_dataset)
save_data_to_file(train_dataset_tensor, 'train_data_modified.txt')

# Process testing data
test_data = process_data_file('test_data.txt',"test_data_after_encode.txt")
test_dataset = CustomDataset(test_data)
test_dataset_tensor = convert_to_tensor(test_dataset)
save_data_to_file(test_dataset_tensor, 'test_data_modified.txt')

# Process validation data
val_data = process_data_file('val_data.txt',"val_data_after_encode.txt")
val_dataset = CustomDataset(val_data)
val_dataset_tensor = convert_to_tensor(val_dataset)
save_data_to_file(val_dataset_tensor, 'val_data_modified.txt')

#start training the data loader
train_dataloader = create_data_loader(train_dataset_tensor, batch_size = 512, shuffle=True) #chnaged train_ for testing 

# Iterate through the training data loader and print at least one label and one one-hot encoding
for data, label in train_dataloader:
    print("Shape of data tensor:", data.shape)
    print("Shape of label tensor:", label.shape)
    
    print("Label:", label[0])
    print("One-hot encoding:")
    print(data[0])
    break

# Define the number of input channels (4 for one-hot encoding) and the number of output classes
input_channels = 4
output_classes = 2  # Assuming binary classification

# Instantiate the mini CNN model
mini_cnn_model = MiniCNN(input_channels, output_classes)

#update for alexnet model
input_channels = 1  # Assuming input sequences have one channel
num_classes = 4 # Example number of classes
model = oneDAlexNet(input_channels, num_classes)

#the following needs to be formatted for 1DAlexnet
#if __name__ == '__main__':
    #seed = 42
    #torch.manual_seed(seed)
    #device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    #model = oneDAlexNet(input_channels, num_classes = 4)  
    #model.to(device)
    #optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
    #loss_fn = F.cross_entropy  # Cross-entropy loss is commonly used for classification tasks

    #EPOCHS = 2
    #for epoch in range(EPOCHS):
       # train(model, device, train_loader, optimizer, loss_fn, epoch)
       # test(model, device, test_loader, loss_fn)

Shape of data tensor: torch.Size([512, 176, 4])
Shape of label tensor: torch.Size([512])
Label: tensor(1.)
One-hot encoding:
tensor([[1., 0., 0., 0.],
        [0., 0., 1., 0.],
        [0., 0., 0., 1.],
        [0., 1., 0., 0.],
        [0., 1., 0., 0.],
        [0., 1., 0., 0.],
        [0., 0., 1., 0.],
        [0., 1., 0., 0.],
        [0., 1., 0., 0.],
        [0., 0., 1., 0.],
        [0., 0., 1., 0.],
        [0., 0., 1., 0.],
        [0., 0., 0., 1.],
        [0., 1., 0., 0.],
        [0., 1., 0., 0.],
        [0., 1., 0., 0.],
        [0., 0., 1., 0.],
        [0., 0., 0., 1.],
        [0., 0., 1., 0.],
        [0., 1., 0., 0.],
        [0., 0., 1., 0.],
        [0., 1., 0., 0.],
        [1., 0., 0., 0.],
        [0., 0., 1., 0.],
        [0., 0., 1., 0.],
        [0., 1., 0., 0.],
        [0., 0., 1., 0.],
        [0., 0., 1., 0.],
        [0., 0., 1., 0.],
        [0., 0., 1., 0.],
        [0., 0., 1., 0.],
        [0., 0., 1., 0.],
        [0., 1., 0., 0.],
        [0., 0., 